# Week 2 Pseudocode Assignment
BIOL 51000 Data Systems in the Life Sciences

November 2022

Victoria Griffin

The following code is an example of 'Importing the HTSeq Library' (requirement 1)

In [8]:
#Import HTSeq library

try:
    from HTSeq import FastqReader, SAM_Reader, GFF_Reader
    from HTSeq import GenomicArray, GenomicInterval
except ValueError:
    print('There is a ValueError') #For some reason my computer is giving me a value error --> something to do with numpy

from matplotlib import pyplot
from numpy import array

print('Success')

There is a ValueError
Success


The following code is an example of 'Importing a FASTQ-format Sequence Including a Header File' (requirement 2)

In [2]:
# Example of a FASTQ File

'''
@r0
GAACGATACCCACCCAACTATCGCCATTCCAGCAT
+
EDCCCBAAAA@@@@?>===<;;9:99987776554
'''
print('Example of a FASTQ file:')
print('@r0')
print('GAACGATACCCACCCAACTATCGCCATTCCAGCAT')
print('+')
print('EDCCCBAAAA@@@@?>===<;;9:99987776554')

Example of a FASTQ file:
@r0
GAACGATACCCACCCAACTATCGCCATTCCAGCAT
+
EDCCCBAAAA@@@@?>===<;;9:99987776554


In [5]:
#Reading a FASTQ File

fileObj = FastqReader(FastqFile) #Using the imported reader class from HTSeq to make an object

# Extract sequence by looping through the FASTQ file
for readSeq in fileObj:
    print(readSeq.name) # header
    print(readSeq.seq) # prints sequence
    print(readSeq.get_reverse_complement()[::-1]) # prints complement of sequence


NameError: name 'FastqReader' is not defined

In [7]:
print('Example of importing FASTQ file will look like this:') #used FASTQ file from above
print('r0')
print('GAACGATACCCACCCAACTATCGCCATTCCAGCAT')
print('CTTGCTATGGGTGGGTTGATAGCGGTAAGGTCGTA')

Example of importing FASTQ file will look like this:
r0
GAACGATACCCACCCAACTATCGCCATTCCAGCAT
CTTGCTATGGGTGGGTTGATAGCGGTAAGGTCGTA


The following code is an example of 'Reading a Genome Alignment File' (requirement 3)

In [ ]:
alignFile = 'examples/EcoliGenomeAlign.sam' #reading a file in the SAM format
chromosomes = set() # variable called chromosomes that defines a set to hold the chromosome identifier

#For creating alignment objects
for alignment in SAM_Reader(alignFile): 
    if alignment.aligned: #checks if alignment exists
        seqRead = alignment.read #read the alignment
        print(seqRead.name) #access sequence name
        print(seqRead.seq) #access sequence 
        
        #This section is for accessing the location that was aligned in the genome sequence
        genomeRegion = alignment.iv #genomic interval
        chromo = genomeRegion.chrom #Find the chromosome number
        strand = genomeRegion.strand #Finds the strand
        start = genomeRegion.start #Identifies start location
        end = genomeRegion.end #Identifies end location
        
        chromosomes.add(chromo) #adds to chromosome set
        print(chromo, start, end, strand) #Prints information found

# set of chromosome identifiers created above are converted into a list and made into a GenomicArray called hitMap
chromosomes = list(chromosomes)
hitMap = GenomicArray(chromosomes, stranded=True, typecode='i') #typecode 'i' means integer

#GenomicArray can be filled with results from SAM alignments
for alignment in SAM_Reader(alignFile):
    if alignment.aligned: #checks to see if alignment is successful
        genomeReion = alignment.iv #Used to set values in genomic array
        
        if genomeRegion.strand == '+':
            hitMap[genomeRegion = 1] # set +1 for leading strand
        else:
            hitMap[genomeRegion = -1] #set -1 for lagging strand, etc

# The section will create objects from scratch and will be used to represent regions of interest
chromo = chromosome[0]
endPoint = 2000000 #intervals are created from position 0 to 2Mbp
plusStrand = GenomicInterval(chromo, 0, endPoint, '+')
minusStrand = GenomicInterval(chromo, 0, endPoint, '-')
bothStrands = GenomicInterval(chromo, 0, endPoint, '.')

# The hitMap is queried with the interval object, which then can be used to convert the results to a list. 
# From the list the results can be plotted in a graph for assessment
pyplot.plot(list(hitMap[plusStrand]))
pyplot.plot(list(hitMap[minusStrand]))
pyplot.show()

The following code is an example of 'Python Code for Annotating a High-Throughput Sequence File' (requirement 4)

In [24]:
remoteFileName = '/Bacteria/Escherichia_coli_536_uid58531/NC_008253.gff' #Access file where gff is found
gffFile = 'examples/EcoliGenomeFeatures.gff' #Access gff
downloadFile(FTP_ROOT+remoteFileName, gffFile) #Download file 

fileObj = GFF_Reader(gffFile) # GFF_Reader can easily access file data

'''Data from file is looped through to obtain GenomicFeatures class objects which 
links genomic location to description of the feature'''

for genomeFeature in fileObj: #For loop

    genomeRegion = genomeFeature.iv #Obtains GeneticFeatures 

    #Information about the chromosome location is stored in variable called 'data'
    data1 = (genomeRegion.chrom,
            genomeRegion.start,
            genomeRegion.end,
            genomeRegion.strand)

    # Will print information from data1
    print('%s %s - %s (%s)' % data)

    # Information about other attributes that also goes along with the location
    # Includes features like introns, exons, genes, etc
    data2 = (genomeFeature.name,
            genomeFeature.type,
            genomeFeature.source)
    
    # Will print information collection from data2
    print('%s %s (%s)' % data)
    
    # Will print information from genomeFeature attribute dictionary
    print(genomeFeature.attr)
    
    '''
    For 'print(genomeFeature.attr)', should print something like:
    
    NC_008253.1 4933963 - 4935385 (+)
    CreC CDA (RefSeq)
    {'locus_tag': 'ECP_4785', 'exon_number': 1, 'product': 'sensory histidine kinase CreC', 'EC_number': '2.7.3.-',
    'note': 'part of a two-component regulatory system with CreB or PhoB%3B involved in catabolic regulation',
    'db_xref': 'GeneID:4190641', 'transl_table': '11', 'protein_id': 'YP_672568.1'}
    '''

    
# Plot features in genomic arrays
# This section will map annotations to a Python representation of the chromosomes

geneMap = GenomicArray('auto', stranded=False, typecode='O') #Will link the GenomicArray directly to annotation objects
genePlot = GenomicArray('auto', stranded=False, typecode='i') #Will hold numbers to plot a graph of gene locations 

# For loop to fill GenomicArray
for genomeFeature in fileObj:
 
    if genomeFeature.type == 'gene': #Will only proceed if feature is of type 'gene'
 
        genomeRegion = genomeFeature.iv 
        geneMap[genomeRegion] = genomeFeature
        genePlot[genomeRegion] = 1 #Set GenePlot to hold value of 1 at feature location will create an intermittent array of values

# For loop to go through what is contained and extract information about it  
    for region, feature in geneMap.steps():
 
        if feature:
 
            data = (feature.name,
                    region.start,
                    region.end,
                    feature.iv.strand)
 
        # Print extracted information
        print('%s: %s - %s (%s)' % data)

# Steps to create a graph of the information      
chromosome = genomeRegion.chrom
region = GenomicInterval(chromosome, 0, 40000, '.')
pyplot.plot(list(genePlot[region])) #Must first change information into a list
pyplot.show()


NameError: name 'downloadFile' is not defined

Week 1 Assigment: Pairwise Alignment

In [9]:
"""Reference matrix used for comparing bases on two sequences → 
match is 8, mismatch is -5, no sequence data is 0"""

DNA = {'G': { 'G': 8, 'C':-5, 'A':-5, 'T':-5, 'N':0 },
       'C': { 'G':-5, 'C': 8, 'A':-5, 'T':-5, 'N':0 },
       'A': { 'G':-5, 'C':-5, 'A': 8, 'T':-5, 'N':0 },
       'T': { 'G':-5, 'C':-5, 'A':-5, 'T': 8, 'N':0 },
       'N': { 'G': 0, 'C': 0, 'A': 0, 'T': 0, 'N':0 }} 

#_________________________________Start Sequence Alignment Code_________________________________________#

# Define function called seqAlign that performs a simple pairwise alignment

def seqAlign(seqA, seqB, simMatrix=DNA, insert=3, extend=4): 
    # gap is penalty of 3, and extension of gap is a penalty of 4

  """Variables numI and numJ represent the length of the rows and columns of the comparison grid → 
     needs to be one length longer than each sequence"""
  numI = len(seqA) + 1
  numJ = len(seqB) + 1

  # Create two matrices. One for holding the scores and one for holding the route taken
  scoreMatrix = [[0] * numJ for x in range(numI)]
  routeMatrix = [[0] * numJ for x in range(numI)]
  
  """I AM NOT SURE IF THIS NEXT SECTION IS CORRECT --> I AM TRYING TO APPLY 0, -3, -9, etc. TO THE OUTER 
     ROW AND COLUMN OF THE MATRIX LIKE WE DID DURING THE LECTURE."""

   # Variables needed to label matrix with correct outer column and row (eg. 0, -3, -6, -9, -12…)
  leftColumn = 0
  topRow = 0

  # Make leftmost column have score start with 0 and decrease by 3 for each iteration
  for i in range(1, numI):
    routeMatrix[i][0] = leftColumn
    leftColumn -= 3
    
  # Make topmost row have score start with 0 and decrease by 3 for each iteration
  for j in range(1, numJ):
    routeMatrix[0][j] = topRow
    topRow -= 3
  
 # Nested loop used to fill in rest of the matrix 
  for i in range(1, numI):
    for j in range(1, numJ):
      
      # Penalty of a gap is 3 like our example from class 
      penalty1 = insert
      penalty2 = insert
      
      #NOT SURE HOW TO FIX THIS THOUGH TO MATCH THE 0,-3, -6….THIS IS WHAT I CAME UP WITH
      # Check to see if previous is a gap
      if routeMatrix[i-1][j] == routeMatrix[0][j]:
        penalty1 = extend
        
      elif routeMatrix[i][j-1] == routeMatrix[i][0]:
        penalty2 = extend
       
       # Look up similarity score of bases for seqA and seqB 
      similarity = simMatrix[ seqA[i-1] ][ seqB[j-1] ]
      
      # Make a list of three possible scores (left-up diagonal, up, and left)
      paths = [scoreMatrix[i-1][j-1] + similarity, # Route 0
                   scoreMatrix[i-1][j] - penalty1, # Route 1
                   scoreMatrix[i][j-1] - penalty2] # Route 2                     
      
      # The best of the three scores are assigned in ‘best’ and the index of best score is assigned to route
      best = max(paths)
      route = paths.index(best)           

      # The scores and indexes for each iteration are saved in the matrix 
      scoreMatrix[i][j] = best
      routeMatrix[i][j] = route

#___________________________________________Working Backwards______________________________________________#
   
   #  Initialize blank lists to hold the optimal sequence alignments for seqA and seqB    
  alignA = []
  alignB = []

  # Assigning variables to implement into while loop below for working backwards
  i = numI-1
  j = numJ-1
  score = scoreMatrix[i][j]


  while i > 0 or j > 0:
    route = routeMatrix[i][j]    

     # Determining which of the three routes had the highest score
    if route == 0: # Diagonal
      alignA.append( seqA[i-1] )
      alignB.append( seqB[j-1] )
      i -= 1
      j -= 1

    elif route == 1: # Gap in seqB
      alignA.append( seqA[i-1] )
      alignB.append( '-' )
      i -= 1      

    elif route == 2: # Gap in seqA
      alignA.append( '-' )
      alignB.append( seqB[j-1] ) 
      j -= 1
  
   # Reverse the alignment lists since we originally stored them backwards, now we want them forwards 
  alignA.reverse()
  alignB.reverse()
  alignA = ''.join(alignA)
  alignB = ''.join(alignB)

  return score, alignA, alignB 




#__________________________________Example Seqeunces Used During Class_____________________________________#


# seqA and seqB are example we used from class
seqA = 'CTTAACT'
seqB = 'CGGATCAT'

score, alignA, alignB = seqAlign(seqA, seqB, DNA) 

# EXPECTED RESULTS
print(score)  #14
print(alignA) # CTTAAC-T
print(alignB) # CGGATCAT





14
CTTAAC-T
CGGATCAT
